In [6]:
%matplotlib inline
import numpy as np
from scipy import stats
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import emcee
import corner
import pdb

(a)

In [53]:
m = np.array([2.23, 2.08, 1.93, 1.78, 1.63, 1.48, 1.33, 1.2, 1.09, 1., 0.93, 0.86, 0.8, 0.74, 0.68, 0.62, 0.56, 0.5])
eps = np.array([6.63, 7.1, 7.36, 7.52, 7.72, 8., 7.98, 7.98, 8.32, 8.5, 8.6, 8.7, 8.83, 8.97, 9.04, 9.13, 9.2, 9.22])

m = 10**(m-1)
eps = 10**(eps-10)

def func(m, c, alpha):
    return c*(m**(-alpha))

popt, pcov = curve_fit(func, m, eps)
c = popt[0]
c_err = np.sqrt(np.diag(pcov))[0]
alpha = popt[1]
alpha_err = np.sqrt(np.diag(pcov))[1]

print('f(m) = %.2f * m ^(-%.2f)' %(c, alpha))

f(m) = 0.04 * m ^(-1.41)


(b)

In [59]:
def ln_prior(theta):
    """
    assume a flat posterior"""
    c = theta[0]
    alpha = theta[1]
    return 0

def ln_likelyhood(theta, m, eps):
    """
    curving fitting on a histogram of the given data"""
    c = theta[0]
    alpha = theta[1]
    xi = c * (m**(-alpha))
    sigma = np.sqrt(eps)
    lnp = -(np.log(sigma) + (xi-eps)**2/(2*sigma**2))
    return np.sum(lnp)

def ln_posterior(theta, m, eps):
    return ln_prior(theta) + ln_likelyhood(theta, m, eps)
    

In [69]:
# generate fake data
def find_theta(m , eps, theta0=[0,0]):
    # use emcee to infer c and a
    steps = 500
    nwalkers = 50
    ndim = 2
    sampler = emcee.EnsembleSampler(nwalkers, ndim, ln_posterior, args=[m, eps])
    pos = [np.random.normal((theta0)) for i in range(nwalkers)]
    sampler.run_mcmc(pos, steps)
    theta = sampler.chain.reshape((-1, ndim))
    
    return theta

theta = find_theta(m , eps)
c2 = np.median(theta[:,0])
c2_err = np.std(theta[:,0])
alpha2 = np.median(theta[:,1])
alpha2_err = np.std(theta[:,1])

print('Using scipy.optimize, we get: f(m) = (%.2f+/-%.2f) * m^(-(%.2f+/-%.2f))' %(c, c_err, alpha, alpha_err))
print('Salpeter 1955 gives the best fit: f(m) = %.2f * m^(-%.2f)' %(0.03, 1.35))
c_sigma = (c-0.03)/c_err
alpha_sigma = np.abs((alpha-1.35)/alpha_err)
print('It is within %.2f sigma for c, within %.2f sigma for alpha\n' %(c_sigma, alpha_sigma))


print('Using emcee, we get: f(m) = (%.2f+/-%.2f) * m^(-(%.2f+/-%.2f))' %(c2, c2_err, alpha2, alpha2_err))
print('Salpeter 1955 gives the best fit: f(m) = %.2f * m^(-%.2f)' %(0.03, 1.35))
c2_sigma = (c-0.03)/c2_err
alpha2_sigma = np.abs((alpha2-1.35)/alpha2_err)
print('It is within %.2f sigma for c, within %.2f sigma for alpha\n' %(c2_sigma, alpha2_sigma))

sig_c = np.abs((c2-c)/np.sqrt(c_err**2+c2_err**2))
print('Comparing scipy.optimize and emcee, they are within %.2f sigma for c'%(sig_c))
sig_alpha = np.abs((alpha2-alpha)/np.sqrt(alpha_err**2+alpha2_err**2))
print('Comparing scipy.optimize and emcee, they are within %.2f sigma for alpha' %(sig_alpha))

Using scipy.optimize, we get: f(m) = (0.04+/-0.00) * m^(-(1.41+/-0.07))
Salpeter 1955 gives the best fit: f(m) = 0.03 * m^(-1.35)
It is within 2.45 sigma for c, within 0.78 sigma for alpha

Using emcee, we get: f(m) = (0.02+/-0.13) * m^(-(1.26+/-1.12))
Salpeter 1955 gives the best fit: f(m) = 0.03 * m^(-1.35)
It is within 0.05 sigma for c, within 0.08 sigma for alpha

Comparing scipy.optimize and emcee, they are within 0.14 sigma for c
Comparing scipy.optimize and emcee, they are within 0.13 sigma for alpha
